In [108]:
import os 
import sys 
import pandas as pd 
import numpy as np
import re
import pickle
import json
import sqlite3
import ast
from pprint import pprint
import datetime
import pdb

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth

In [3]:
with open('../credentials/spotify_creds.pkl', 'rb') as hnd:
    credentials = pickle.load(hnd)

In [4]:
os.environ.update(credentials)

In [5]:
scope = "user-read-recently-played"

In [6]:
spotify = spotipy.Spotify(client_credentials_manager= SpotifyOAuth(scope=scope
                                                                   , username='malchemist02'))

In [7]:
db_location = 'data/listening_history.db'

In [43]:
con = sqlite3.connect(db_location)
cursor = con.cursor()
cursor.execute('SELECT name FROM sqlite_master WHERE type=\'table\' ORDER BY name')
tables = cursor.fetchall()
tables = [x[0] for x in tables]
tables

['Listening_History']

In [42]:
# cursor.execute("drop table artists_info")
# con.commit()
# con.close()

In [8]:
db = pd.read_sql('select distinct artist_id from Listening_History', con)
db

,artist_id
0,spotify:artist:2M72oi0t6i7mADfvCtekOM
1,spotify:artist:5aAlzehdUM14I4ppq24Xob
2,spotify:artist:50JJSqHUf2RQ9xsHs0KMHg
3,spotify:artist:6jTnHxhb6cDCaCu4rdvsQ0
4,spotify:artist:3ApUX1o6oSz321MMECyIYd
...,...
1148,spotify:artist:1ZwdS5xdxEREPySFridCfh
1149,spotify:artist:1Ffb6ejR6Fe5IamqA5oRUF
1150,spotify:artist:0XK6kT7xcZAlcYrNjOgzJe
1151,spotify:artist:0iAJZKKcpZEiIw5hRABr91


In [11]:
existing_artist = pd.read_sql('select distinct artist_id from artists_info', con)
existing_artist

,artist_id
0,spotify:artist:4TjR4K8rD5CqqqU59V6aGr
1,spotify:artist:5dCuFngSPyOOnTAvrC7v2s
2,spotify:artist:2lolQgalUvZDfp5vvVtTYV
3,spotify:artist:7KHQppBj7ea6qRp4NsTN1u
4,spotify:artist:3K2Srho6NCF3o9MswGR76H
...,...
1057,spotify:artist:7vXDAI8JwjW531ouMGbfcp
1058,spotify:artist:69UypehHabb68utzfjAVlV
1059,spotify:artist:2LCcy9CZWwZ7Vvykt8IVVq
1060,spotify:artist:3SCOuAxngTC1yGjKMcIPEd


In [17]:
merged = pd.merge(db, existing_artist, on ='artist_id', how= 'outer', indicator=True)
print(merged.head())
merged['_merge'].value_counts()

                               artist_id _merge
0  spotify:artist:2M72oi0t6i7mADfvCtekOM   both
1  spotify:artist:5aAlzehdUM14I4ppq24Xob   both
2  spotify:artist:50JJSqHUf2RQ9xsHs0KMHg   both
3  spotify:artist:6jTnHxhb6cDCaCu4rdvsQ0   both
4  spotify:artist:3ApUX1o6oSz321MMECyIYd   both


both          1062
left_only       91
right_only       0
Name: _merge, dtype: int64

In [23]:
artists_all = merged[merged['_merge'] == 'left_only']['artist_id'].values.tolist()
artists_all = list(set(artists_all))
print("Number of artists: ", len(artists_all))
artists_all[:5]

Number of artists:  91


['spotify:artist:4RyILBSzO6HaIMayAo6M4W',
 'spotify:artist:7G2JO0YKokgwGDFCdqBIxQ',
 'spotify:artist:4igDSX1kgfWbVTDCywcBGm',
 'spotify:artist:72uoxerTvAd7x3cbfYmNc8',
 'spotify:artist:4ERtgeBbWRkFzIz6LaFCeY']

In [32]:
artists_all = db['artist_id'].values.tolist()
artists_all = list(set(artists_all))
print("Number of artists: ", len(artists_all))
artists_all[:5]

Number of artists:  1153


['spotify:artist:7h8ja4JSORo2sXJPmCXRxa',
 'spotify:artist:3wWtfT7S2uVJJ3hGZlOLkZ',
 'spotify:artist:1GKYNY4rIPnOuTfC0J1IWw',
 'spotify:artist:2kHxkdiKCSnHMkhIgFBZaI',
 'spotify:artist:4BwjcAgEbDOtlyFDyKJkC2']

In [33]:
all_rslts = []
for x in range(1+int(len(artists_all)/50)):
    start = x*50
    end = (x+1)*50
    print("Start Num: ", start, " End: ", end, "Started at ", datetime.datetime.now())
    temp_rslts = spotify.artists(artists_all[start:end])
    all_rslts.append(temp_rslts)

Start Num:  0  End:  50 Started at  2021-01-21 20:55:29.535893
Start Num:  50  End:  100 Started at  2021-01-21 20:55:29.602956
Start Num:  100  End:  150 Started at  2021-01-21 20:55:29.668201
Start Num:  150  End:  200 Started at  2021-01-21 20:55:29.734592
Start Num:  200  End:  250 Started at  2021-01-21 20:55:29.819053
Start Num:  250  End:  300 Started at  2021-01-21 20:55:29.896921
Start Num:  300  End:  350 Started at  2021-01-21 20:55:29.954404
Start Num:  350  End:  400 Started at  2021-01-21 20:55:30.012553
Start Num:  400  End:  450 Started at  2021-01-21 20:55:30.078761
Start Num:  450  End:  500 Started at  2021-01-21 20:55:30.173450
Start Num:  500  End:  550 Started at  2021-01-21 20:55:30.255980
Start Num:  550  End:  600 Started at  2021-01-21 20:55:30.312300
Start Num:  600  End:  650 Started at  2021-01-21 20:55:30.386584
Start Num:  650  End:  700 Started at  2021-01-21 20:55:30.461116
Start Num:  700  End:  750 Started at  2021-01-21 20:55:30.551635
Start Num:  75

In [34]:
all_artist_info = []
for x in all_rslts:
    all_artist_info.extend(x['artists'])
print(len(all_artist_info))

1153


In [35]:
def parse_artist_rslt(artist_dict):
    out = {}
    out['artist_name'] = artist_dict.get('name')
    out['uri'] = artist_dict.get('id')
    out['artist_id'] = 'spotify:artist:' + out['uri']
    out['followers'] = artist_dict.get('followers').get('total')
    out['genres'] = str(artist_dict.get('genres'))
    out['popularity'] = artist_dict.get('popularity')
    out['pull_date'] = datetime.datetime.now().date()
    
    out_series = pd.Series(out)
    return(out_series)

In [36]:
artist_series = [parse_artist_rslt(x) for x in all_artist_info]
artist_df = pd.concat(artist_series, axis=1).T
artist_df['pull_date'] = artist_df['pull_date'].astype('str')
artist_df['genres'] = artist_df['genres'].astype('str')
artist_df.head()

,artist_name,uri,artist_id,followers,genres,popularity,pull_date
0,Immortal Technique,7h8ja4JSORo2sXJPmCXRxa,spotify:artist:7h8ja4JSORo2sXJPmCXRxa,371585,"['conscious hip hop', 'east coast hip hop', 'g...",56,2021-01-21
1,Judah & the Lion,3wWtfT7S2uVJJ3hGZlOLkZ,spotify:artist:3wWtfT7S2uVJJ3hGZlOLkZ,235849,"['banjo', 'folk-pop', 'indiecoustica', 'modern...",63,2021-01-21
2,Audrey Assad,1GKYNY4rIPnOuTfC0J1IWw,spotify:artist:1GKYNY4rIPnOuTfC0J1IWw,116080,"['anthem worship', 'ccm', 'christian alternati...",57,2021-01-21
3,Kristen Bell,2kHxkdiKCSnHMkhIgFBZaI,spotify:artist:2kHxkdiKCSnHMkhIgFBZaI,63329,"['disney', 'hollywood', 'show tunes']",70,2021-01-21
4,Tory Rines,4BwjcAgEbDOtlyFDyKJkC2,spotify:artist:4BwjcAgEbDOtlyFDyKJkC2,854,[],45,2021-01-21


In [37]:
artist_df.columns

Index(['artist_name', 'uri', 'artist_id', 'followers', 'genres', 'popularity',
       'pull_date'],
      dtype='object')

In [46]:
df2 = pd.read_sql('select * from artists_info', con)
df2.head()

,artist_name,uri,artist_id,followers,genres,popularity,pull_date
0,Immortal Technique,7h8ja4JSORo2sXJPmCXRxa,spotify:artist:7h8ja4JSORo2sXJPmCXRxa,371585,"['conscious hip hop', 'east coast hip hop', 'g...",56,2021-01-21
1,Judah & the Lion,3wWtfT7S2uVJJ3hGZlOLkZ,spotify:artist:3wWtfT7S2uVJJ3hGZlOLkZ,235849,"['banjo', 'folk-pop', 'indiecoustica', 'modern...",63,2021-01-21
2,Audrey Assad,1GKYNY4rIPnOuTfC0J1IWw,spotify:artist:1GKYNY4rIPnOuTfC0J1IWw,116080,"['anthem worship', 'ccm', 'christian alternati...",57,2021-01-21
3,Kristen Bell,2kHxkdiKCSnHMkhIgFBZaI,spotify:artist:2kHxkdiKCSnHMkhIgFBZaI,63329,"['disney', 'hollywood', 'show tunes']",70,2021-01-21
4,Tory Rines,4BwjcAgEbDOtlyFDyKJkC2,spotify:artist:4BwjcAgEbDOtlyFDyKJkC2,854,[],45,2021-01-21


In [44]:
artist_df.to_sql('Artists_Info', con, index=False, if_exists='append')

In [47]:
artist_df.sort_values('popularity', ascending=False).head(20)

,artist_name,uri,artist_id,followers,genres,popularity,pull_date
704,Bad Bunny,4q3ewBCX7sLwd24euuV69X,spotify:artist:4q3ewBCX7sLwd24euuV69X,28984059,"['latin', 'reggaeton', 'trap latino']",100,2021-01-21
140,Ariana Grande,66CXWjxzNUsdJxJ2JdwvnR,spotify:artist:66CXWjxzNUsdJxJ2JdwvnR,56414774,"['dance pop', 'pop', 'post-teen pop']",98,2021-01-21
679,Taylor Swift,06HL4z0CvFAxyc27GXpf02,spotify:artist:06HL4z0CvFAxyc27GXpf02,35853019,"['dance pop', 'pop']",98,2021-01-21
988,Juice WRLD,4MCBfE4596Uoi2O4DtmEMz,spotify:artist:4MCBfE4596Uoi2O4DtmEMz,15061857,"['chicago rap', 'melodic rap']",98,2021-01-21
819,Drake,3TVXtAsR1Inumwj472S9r4,spotify:artist:3TVXtAsR1Inumwj472S9r4,52148805,"['canadian hip hop', 'canadian pop', 'hip hop'...",97,2021-01-21
612,Justin Bieber,1uNFoZAHBGtllmzznpCI3s,spotify:artist:1uNFoZAHBGtllmzznpCI3s,41553536,"['canadian pop', 'pop', 'post-teen pop']",96,2021-01-21
1126,J Balvin,1vyhD5VmyZ7KMfW5gqLgo5,spotify:artist:1vyhD5VmyZ7KMfW5gqLgo5,25557459,"['latin', 'reggaeton', 'reggaeton colombiano',...",96,2021-01-21
510,BTS,3Nrfpe0tUJi4K4DXYWgMUX,spotify:artist:3Nrfpe0tUJi4K4DXYWgMUX,27730580,"['k-pop', 'k-pop boy group']",96,2021-01-21
192,The Weeknd,1Xyo4u8uXC1ZmMpatF05PJ,spotify:artist:1Xyo4u8uXC1ZmMpatF05PJ,27776847,"['canadian contemporary r&b', 'canadian pop', ...",95,2021-01-21
514,Pop Smoke,0eDvMgVFoNV3TpwtrVCoTj,spotify:artist:0eDvMgVFoNV3TpwtrVCoTj,3649080,['brooklyn drill'],94,2021-01-21


In [53]:
all_genres_raw = [ast.literal_eval(row) for row in artist_df['genres']] 
all_genres = []
_ = [all_genres.extend(x) for x in all_genres_raw]
all_genres[:3]

['conscious hip hop', 'east coast hip hop', 'gangster rap']

In [55]:
from collections import Counter
counter = Counter(all_genres)
counter.most_common(10)

[('pop', 194),
 ('dance pop', 119),
 ('latin', 91),
 ('background piano', 85),
 ('tropical', 84),
 ('post-teen pop', 82),
 ('pop rap', 76),
 ('modern rock', 67),
 ('pop dance', 67),
 ('contemporary country', 66)]

## Playlists

In [64]:
demo_df = pd.read_sql('select distinct playlist_id from listening_history', con)
demo_df

,playlist_id
0,spotify:playlist:5UJhGomjPYIXhtkJiTXMYR
1,spotify:playlist:2AP3UU4TfEmHcWWw0LVlOZ
2,None
3,spotify:playlist:5sEM3oiygrXyRP2USzyDGs
4,spotify:playlist:37i9dQZF1DX4sWSpwq3LiO
...,...
161,spotify:playlist:37i9dQZF1E8CUVODKSY7ca
162,spotify:playlist:37i9dQZEVXbivEXjvLnLc7
163,spotify:artist:18qC8mrcJ9ZjChRDPvpadi
164,spotify:artist:6UlNIFEuWlBqb4TvlVCekq


In [68]:
len('spotify:playlist:')

17

In [72]:
playlists = list(demo_df['playlist_id'].unique())
playlists.remove(None)
playlists = [x for x in playlists if (x[:17] == 'spotify:playlist:')]
len(playlists)

107

In [75]:
bad_playlist_ids = []
all_playlists = []
for x in playlists:
    try:
        rslt = spotify.playlist(x)
        all_playlists.append(rslt)
    except Exception as e:
        bad_playlist_ids.append(x)
spot_playlists = all_playlists[0]
spot_playlists

Expected id of type playlist but found type 2AP3UU4TfEmHcWWw0LVlOZ spotify:playlist:2AP3UU4TfEmHcWWw0LVlOZ:recommended
HTTP Error for GET to https://api.spotify.com/v1/playlists/recommended returned 404 due to Invalid playlist Id
Expected id of type playlist but found type 7MdgBbt3ZyLDSeoPTEUy4o spotify:playlist:7MdgBbt3ZyLDSeoPTEUy4o:recommended
HTTP Error for GET to https://api.spotify.com/v1/playlists/recommended returned 404 due to Invalid playlist Id


{'collaborative': False,
 'description': '',
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/5UJhGomjPYIXhtkJiTXMYR'},
 'followers': {'href': None, 'total': 0},
 'href': 'https://api.spotify.com/v1/playlists/5UJhGomjPYIXhtkJiTXMYR?additional_types=track',
 'id': '5UJhGomjPYIXhtkJiTXMYR',
 'images': [{'height': 640,
   'url': 'https://mosaic.scdn.co/640/ab67616d0000b27353e13a9e568364f6b7c53f1fab67616d0000b273d8eb78c0aa8a1dff08bb5890ab67616d0000b273e937b8b72a5be0e2eba5673dab67616d0000b273fddf6509ff2b9c94a793e49a',
   'width': 640},
  {'height': 300,
   'url': 'https://mosaic.scdn.co/300/ab67616d0000b27353e13a9e568364f6b7c53f1fab67616d0000b273d8eb78c0aa8a1dff08bb5890ab67616d0000b273e937b8b72a5be0e2eba5673dab67616d0000b273fddf6509ff2b9c94a793e49a',
   'width': 300},
  {'height': 60,
   'url': 'https://mosaic.scdn.co/60/ab67616d0000b27353e13a9e568364f6b7c53f1fab67616d0000b273d8eb78c0aa8a1dff08bb5890ab67616d0000b273e937b8b72a5be0e2eba5673dab67616d0000b273fddf6509ff2b9c94a793

In [76]:
len(bad_playlist_ids)

2

In [115]:
def parse_all_tracks(track_items):
    all_tracks = []
    for track in track_items:
        out_dict = {}
        out_dict['added_at'] = track['added_at']
        track_details = track['track']
        out_dict['name'] = track_details['name']
        out_dict['artist'] = track_details['artists'][0]['name']
        out_dict['duration_min'] = track_details['duration_ms']/60000
        out_dict['popularity'] = track_details['popularity']
        out_dict['uri'] = track_details['uri']
        out_dict['release_date'] = track_details['album']['release_date']

        all_tracks.append(out_dict)
    df = pd.DataFrame(all_tracks).sort_values('added_at')|
    return(df)

In [127]:
def parse_playlist_rslt(spot_playlists):
    spot_details = {}
    spot_details['name'] = spot_playlists['name']
    spot_details['# of Tracks'] = len(spot_playlists['tracks']['items'])
    spot_details['pull_date'] = str(datetime.datetime.now().date())
    tracks_df = parse_all_tracks(spot_playlists['tracks']['items'])
    spot_details['Time (min)'] = np.round(tracks_df['duration_min'].sum(), 2)
    spot_details['Avg Popularity'] = np.round(tracks_df['popularity'].mean(), 2)
    
    # Temp 
    spot_details['track_df'] = tracks_df
    return(spot_details)

In [117]:
rslts = [parse_playlist_rslt(x) for x in all_playlists]
df = pd.DataFrame(rslts)
df

,name,# of Tracks,pull_date,Time (min),Avg Popularity,track_df
0,Decent Salsa,63,2021-01-22,289.52,41.95,added_at ...
1,summer 2020,36,2021-01-22,118.48,60.75,added_at ...
2,Luke Combs – Forever After All + More Hits,50,2021-01-22,176.68,60.60,added_at ...
3,Peaceful Piano,100,2021-01-22,267.33,64.48,added_at name ...
4,Confidence Boost,70,2021-01-22,227.28,68.11,added_at ...
...,...,...,...,...,...,...
100,¡Viva Latino!,50,2021-01-22,168.43,84.88,added_at ...
101,Jan 2021,10,2021-01-22,33.77,64.20,added_at ...
102,El Mismo Aire - con Pablo Alborán Radio,50,2021-01-22,166.67,72.12,added_at ...
103,Esta Vida Me Encanta Radio,50,2021-01-22,168.66,61.18,added_at ...


In [123]:
df['track_df'][1]

,added_at,name,artist,duration_min,popularity,uri,release_date
0,2020-06-28T22:10:17Z,that's just how it goes,ROLE MODEL,2.773333,62,spotify:track:4J4rJEaVpgr2tHwX6Jd13H,2019-11-13
1,2020-07-12T04:47:22Z,Coffee (feat. Marc E. Bassy),Quinn XCII,3.000883,7,spotify:track:5SUUALLSTMpyrnnVUUA5wG,2020-05-29
2,2020-07-12T00:37:50Z,Monsters (feat. blackbear),All Time Low,2.901117,78,spotify:track:0tyR7Bu9P086aWBFZ4QJoo,2020-04-03
3,2020-07-12T00:38:12Z,Bang!,AJR,2.847633,80,spotify:track:53BHUFdQphHiZUUG3nx9zn,2020-02-12
4,2020-07-12T12:26:41Z,The Difference,Flume,2.327700,73,spotify:track:4nlvKIIetOWGIMyhjQXgOZ,2020-03-11
5,2020-07-12T12:39:21Z,Chicken Tenders,Dominic Fike,2.840083,61,spotify:track:3D8sT8D3f5egWSQnF0fbqu,2020-06-26
6,2020-07-21T04:12:14Z,Alaska,Little Hurt,3.144633,60,spotify:track:0sSoNYwZLoE4gLAKIG4uYL,2020-05-01
7,2020-07-21T04:12:36Z,Sunday Best,Surfaces,2.642850,85,spotify:track:1Cv1YLb4q0RzL6pybtaMLo,2019-01-06
8,2020-07-27T21:25:12Z,Who’s Gonna Love Me Now,Cold War Kids,2.853767,48,spotify:track:3OAiJ8YdnNr2zus6CwT3y2,2020-02-11
9,2020-07-27T22:48:41Z,Heat Waves,Glass Animals,3.980083,82,spotify:track:6CDzDgIUqeDY5g8ujExx2f,2020-06-29


In [124]:
df['track_df'][0]['artist'].value_counts().head()

Rey Ruiz               5
Tito Nieves            4
Luis Enrique           4
LA INDIA               4
Gilberto Santa Rosa    3
Name: artist, dtype: int64

In [126]:
df['track_df'][0].sort_values('added_at')[:2]

,added_at,name,artist,duration_min,popularity,uri,release_date
0,2020-04-19T01:49:51Z,Mas de Mi,Tony Succar,5.122000,43,spotify:track:6VZ9ABcY9QeQSxOjZwLrVv,2019-05-24
1,2020-05-01T17:32:22Z,Tres Días,Havana D’Primera,5.351117,49,spotify:track:33hjoPb45TjMp0TGYDawsv,2018-04-10
2,2020-05-18T17:40:52Z,Luna Negra,Rey Ruiz,4.734217,52,spotify:track:09Prj3ivNAC9Y80cP4eKnm,1992
3,2020-05-18T17:41:18Z,Amor De Una Noche (feat. Julio Voltio) - Remix...,N'Klabe,4.455767,54,spotify:track:6B9r2hH46YbSEvnivmeF5C,2005-10-18
4,2020-05-24T15:43:49Z,Abre Que Voy,Miguel Enriquez,4.379100,50,spotify:track:3ihMiG7GPZdsWJcf7Tyevw,2007-09-18
